# Tutorial analysis

In this tutorial, you'll run your first analysis with Lacuna.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m-petersen/lacuna/blob/main/docs/tutorials/getting-started.ipynb)


In [ ]:
# @title 🚀 Setup Environment (Run this first!)
# @markdown This cell installs MRtrix3, Python 3.11, and Lacuna on Google Colab.
# @markdown **Note:** The runtime will automatically restart during this process to apply updates.
# @markdown If the cell stops, **please run it one more time** until you see "✅ Installation Complete".

import os
import sys
import time

# Detect if running on Colab
if 'google.colab' in sys.modules:
    try:
        # 1. Check if Lacuna is already working
        import lacuna
        print("✅ Installation Complete. Lacuna is ready.")
        
    except ImportError:
        print("🐢 Lacuna not found. Starting installation (approx 60s)...")
        
        # --- STEP A: Conda Setup ---
        # condacolab.install() might trigger a restart on its own.
        # If the cell stops here, just run it again!
        if not os.path.exists("/usr/local/conda"):
            !pip install -q condacolab
            import condacolab
            condacolab.install()
        
        # --- STEP B: Install MRtrix3 & Python 3.11 ---
        print("📦 Installing MRtrix3 and dependencies...")
        !mamba install -y -c mrtrix3 -c conda-forge mrtrix3 python=3.11
        
        # --- STEP C: Install Lacuna from GitHub ---
        print("📦 Installing Lacuna...")
        !pip install git+https://github.com/m-petersen/lacuna
        
        print("🔄 Dependencies updated. Restarting runtime to load new versions...")
        print("⚠️ Execution will stop here. Please RUN THIS CELL ONE MORE TIME to finish!")
        
        # Give a moment for prints to show
        time.sleep(1)
        
        # Force Colab to restart the kernel to load the new Numpy/Dateutil
        os.kill(os.getpid(), 9)

else:
    print("Not running on Colab. Assuming local environment is set up.")

In [1]:
print("Hello, World!")

Hello, World!


In [2]:
! lacuna --help

usage: lacuna [-h] [--version] <command> ...

Lacuna: Lesion Network Mapping Analysis v0.0.1.dev282

options:
  -h, --help  show this help message and exit
  --version   show program's version number and exit

commands:
  Use 'lacuna <command> --help' for more information.

  <command>
    fetch     Download and setup connectomes
    run       Run lesion network mapping analyses
    collect   Aggregate parcelstats across subjects
    info      Display available resources (atlases, connectomes)
    bidsify   Convert NIfTI files to BIDS format
    tutorial  Setup tutorial data for learning Lacuna

Commands:
  fetch     Download and setup connectomes for analysis
  run       Run lesion network mapping analyses
  collect   Aggregate results across subjects
  info      Display available resources (atlases, connectomes)
  tutorial  Setup tutorial data for learning Lacuna

Examples:
  lacuna tutorial ./my_tutorial
  lacuna fetch gsp1000 --api-key \$DATAVERSE_API_KEY
  lacuna run rd /bids /out